# Import packages and own modules

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

import holidays

import sys
sys.path.append('../src')
from helperFunctions import *

%matplotlib inline

# 1. Load input data
## 1.1 Load and merge

In [3]:
train = pd.read_csv("../data/train.csv")
store = pd.read_csv("../data/store.csv")
merged_train_store = pd.merge(store, train, on='Store', how='left')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
my_features = ['Sales_per_customer', 'CompetitionDistance', 'Customer_per_store', 
               'Store_menc', 'CompetitionOpenSincePeriod', 
               'Month', 'Year', 'DayOfWeek', 'WeekOfYear', 'DayOfMonth', 
               'StoreType_b', 'StoreType_c', 'StoreType_d', 
               'Promo2SincePeriod', 'Promo2', 'Promo', 
               'HasCompetitor', 'CompetitionOpenSincePeriod', 
               'Assortment', 'CompetitionDistance']


existing_features = list(set.intersection(set(merged_train_store.columns),set(my_features)))
new_features = list(set(my_features).difference(set(existing_features)))

new_features

data_train = merged_train_store.loc[:, existing_features].dropna()
for feat in new_features:
    data_train[feat] = 0

data_train

,Assortment,Promo,Promo2,DayOfWeek,CompetitionDistance,CompetitionOpenSincePeriod,HasCompetitor,DayOfMonth,StoreType_d,StoreType_b,Store_menc,Customer_per_store,StoreType_c,Month,Promo2SincePeriod,WeekOfYear,Year,Sales_per_customer
0,a,0.0,0,2.0,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,a,0.0,0,3.0,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,a,0.0,0,4.0,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,a,0.0,0,5.0,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,a,0.0,0,6.0,1270.0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618465,c,0.0,1,4.0,5350.0,0,0,0,0,0,0,0,0,0,0,0,0,0
618469,c,1.0,1,1.0,5350.0,0,0,0,0,0,0,0,0,0,0,0,0,0
618470,c,1.0,1,2.0,5350.0,0,0,0,0,0,0,0,0,0,0,0,0,0
618471,c,1.0,1,3.0,5350.0,0,0,0,0,0,0,0,0,0,0,0,0,0
